In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
# CNN 모델 정의
def build_CNN_classifier(x):
    # MNIST 데이터를 3차원 형태로 reshape합니다. MNIST 데이터는 grayscale 이미지기 때문에
    # 3번째차원(컬러채널)의 값은 1입니다.
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    
    # 첫 번째 Convolution Layer
    # 5*5 Kernel Size를 가진 32개의 Filter를 적용
    # 28*28*1 -> 28*28*32
    W_conv1 = tf.Variable(tf.truncated_normal(shape=[5, 5, 1, 32], stddev=5e-2))
    b_conv1 = tf.Variable(tf.constant(0.1, shape=[32]))
    h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1)
    
    # 첫 번째 Pooling Layer
    # Max Pooling을 이용해서 이미지의 크기를 1/2로 downsample
    # 28*28*32 -> 14*14*32
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    # 두 번째 Convolution Layer
    # 5*5 Kernel Size를 가진 64개의 Filter를 적용
    # 14*14*32 -> 14*14*64
    W_conv2 = tf.Variable(tf.truncated_normal(shape=[5, 5, 32, 64], stddev=5e-2))
    b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
    h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2) 
    
    # 두 번째 Pooling Layer
    # Max Pooling을 이용해서 이미지의 크기를 1/2로 downsample
    # 14*14*64 -> 7*7*64
    h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    # Fully Connected Layer
    # 7*7 크기를 가진 64개의 activation map을 1024개의 특징들로 변환
    # 7*7*64(3136) -> 1024
    W_fc1 = tf.Variable(tf.truncated_normal(shape=[7*7*64, 1024], stddev=5e-2))
    b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    
    # Output Layer
    # 1024개의 특징(feature)을 10개의 클래스-one-hot encoding으로 표현된 숫자 0-9-로 변환
    # 1024 -> 10
    W_output = tf.Variable(tf.truncated_normal(shape=[1024, 10], stddev=5e-2))
    b_output = tf.Variable(tf.constant(0.1, shape=[10]))
    logits = tf.matmul(h_fc1, W_output) + b_output
    y_pred = tf.nn.softmax(logits)
    
    return y_pred, logits

In [3]:
#인풋,아웃풋 데이터를 받기위한 플레이스 홀더를 정의
x=tf.placeholder(tf.float32,shape=[None,784])
y=tf.placeholder(tf.float32,shape=[None,10])

#Convolutional Neural Networks CNN을 정의
y_pred, logits=build_CNN_classifier(x)

#Cross Entropy를 손실함수로 정의하고 옵티마이저를 정의
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=logits))
train_step=tf.train.AdamOptimizer(1e-4).minimize(loss)

#정확도를 계산하는 연산을 추가
correct_prediction=tf.equal(tf.argmax(y_pred,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [4]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(10000):
        batch=mnist.train.next_batch(50)
        
        if i % 100 == 0:
            train_accuracy=accuracy.eval(feed_dict={x:batch[0], y:batch[1]})
        print("epoch : %d train_accuracy : %f" % (i, train_accuracy))
        
        sess.run([train_step],feed_dict={x:batch[0], y:batch[1]})
        
    print("test_Acc : %f"%(accuracy.eval(feed_dict={x:mnist.test.images, y:mnist.test.labels})))

epoch : 0 train_accuracy : 0.060000
epoch : 1 train_accuracy : 0.060000
epoch : 2 train_accuracy : 0.060000
epoch : 3 train_accuracy : 0.060000
epoch : 4 train_accuracy : 0.060000
epoch : 5 train_accuracy : 0.060000
epoch : 6 train_accuracy : 0.060000
epoch : 7 train_accuracy : 0.060000
epoch : 8 train_accuracy : 0.060000
epoch : 9 train_accuracy : 0.060000
epoch : 10 train_accuracy : 0.060000
epoch : 11 train_accuracy : 0.060000
epoch : 12 train_accuracy : 0.060000
epoch : 13 train_accuracy : 0.060000
epoch : 14 train_accuracy : 0.060000
epoch : 15 train_accuracy : 0.060000
epoch : 16 train_accuracy : 0.060000
epoch : 17 train_accuracy : 0.060000
epoch : 18 train_accuracy : 0.060000
epoch : 19 train_accuracy : 0.060000
epoch : 20 train_accuracy : 0.060000
epoch : 21 train_accuracy : 0.060000
epoch : 22 train_accuracy : 0.060000
epoch : 23 train_accuracy : 0.060000
epoch : 24 train_accuracy : 0.060000
epoch : 25 train_accuracy : 0.060000
epoch : 26 train_accuracy : 0.060000
epoch : 27 